In [ ]:
%pip install ultralytics

In [1]:
import os
import random
import shutil
from ultralytics import YOLO
from glob import glob

In [ ]:
class LabelUtilsYOLOv8:

    # Перемешиваем нарезанные картинки(т.е. меняем местами названиям картинок)
    def create_shuffled_images_folder(self):
        if not os.path.exists("shuffled_images"):
            os.mkdir("shuffled_images")

        image_files = [f for f in os.listdir("images") if f.endswith(".jpg")]
        random.shuffle(image_files)

        for img in image_files:
            new_img_name = f"shuffled_images/img_{len(os.listdir('shuffled_images'))}.jpg"
            os.rename(f"images/{img}", new_img_name)
    
    # Перемещаем перемешанный картинки и их лейблы в папку obj, архивируем её
    def create_labeled_images_zip_file(self):
        if not os.path.exists("obj"):
            os.mkdir("obj")

        for txt_file in glob("shuffled_images/*.txt"):
            base_name = os.path.splitext(os.path.basename(txt_file))[0]
            shutil.move(txt_file, f"obj/{base_name}.txt")
            shutil.move(f"shuffled_images/{base_name}.jpg", f"obj/{base_name}.jpg")

        shutil.make_archive('yolov8/obj', 'zip', 'obj')

    # Делаем конфиг для YOLOv8
    def update_config_files(self, classes):
        data_yaml_content = f"names: {classes}\nnc: {len(classes)}\ntrain: ../obj\nval: ../obj"
        with open("./yolov8/data.yaml", "w") as file:
            file.write(data_yaml_content)





In [ ]:
# Обрабатываем датасет и делаем конфиг
label_utils = LabelUtilsYOLOv8()
label_utils.create_shuffled_images_folder()
label_utils.create_labeled_images_zip_file()
label_utils.update_config_files(['rat'])  # В аргументе имена классов которые залейблили